In [1]:
from general_imports import *

In [2]:
df = pd.read_csv("data/products_cellphones_6_3.csv")
df.head(n=5)

,ID,Title,Link
0,0,iPhone 15 Pro Max 256GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-15-pro-max.html
1,1,iPhone 13 128GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-13.html
2,2,iPhone 14 Pro Max 128GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-14-pro-max.html
3,3,iPhone 15 128GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-15.html
4,4,iPhone 14 128GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-14.html


Perform crawling for each link and extract the data from the page.

In [176]:
def init_driver(url):
    service = Service(executable_path="chromedriver.exe")

    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    driver = webdriver.Chrome(
        service=service,
        options=options)
    
    driver.get(url)
    driver.maximize_window()

    return driver

def move_to_element(driver, element):
    action = ActionChains(driver)
    element = driver.find_element(By.CLASS_NAME, element)
    action.move_to_element(element)
    
    action.perform()

def save_link_to_product(driver, link, name):
    # brand name
    brand_name = name.split(" ")[0]

    if link == "https://cellphones.com.vn/dien-thoai-oppo-reno-11-f.html":
        move_to_element(driver, 'cps-container')
        time.sleep(5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # original price 
        span_tag = soup.find('span', {'class': 'text-price'})
        original_price = span_tag.strong.get_text().replace("đ", "").replace(".", "").strip()

        # product info
        ul_element = soup.find('ul', {'class': 'technical-content'})
        li_elements = ul_element.find_all('li')

        data_info = {}
        
        for li in li_elements: 
            p_tag = li.find('p')
            div_tag = li.find('div')

            key = p_tag.get_text()
            value = div_tag.get_text()

            data_info[key] = value
        
        result = [name, brand_name, original_price, None, None, None, data_info, link]
        print(result)
        driver.close()
        return result

    # handle for product_info
    move_to_element(driver, 'box-warranty-info')
    time.sleep(1)
    driver.execute_script('window.scrollBy(0, 1000);')
    time.sleep(1)
    move_to_element(driver, 'block-same-product')
    driver.execute_script('window.scrollBy(0, 500);')


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # original price (tpt---sale-price) // product__price--show
    try:
        original_price = soup.find_all('p', {"class": 'tpt---sale-price'})
        if len(original_price) > 0:
            original_price = soup.find_all('p', {"class": 'tpt---sale-price'})[1].get_text()\
                .replace("đ", "").replace(".", "").strip()
        else: 
             original_price = soup.find('p', {"class": 'product__price--show'}).get_text()\
                .replace("đ", "").replace(".", "").strip()
    except:
        original_price = None

    # original price (tpt---price) // product__price--through
    try:
        discount_price = soup.find_all('p', {"class": 'tpt---price'})
        if len(discount_price) > 0:
            discount_price = soup.find_all('p', {"class": 'tpt---price'})[0].get_text()\
                .replace("đ", "").replace(".", "").strip()
        else:
            discount_price = soup.find('p', {"class": 'product__price--through'}).get_text()\
                .replace("đ", "").replace(".", "").strip()
    except:
        discount_price = None
        
    # star_rating
    try:
        star_rating = soup.find('p', {"class": 'title is-4 m-0 p-0'}).get_text()
    except AttributeError:
        star_rating = None

    # rewview_count
    try:
        review_count = soup.find('p', {"class": 'boxReview-score__count'})
        review_count = review_count.find('strong').get_text()
    except AttributeError:
        review_count = None

    # product info
    ul_element = soup.find('ul', {'class': 'technical-content'})
    li_elements = ul_element.find_all('li')

    data_info = {}

    for li in li_elements: 
        p_tag = li.find('p')
        div_tag = li.find('div')

        key = p_tag.get_text(strip=True)
        value = div_tag.get_text(strip=True)

        data_info[key] = value

    result = [name, brand_name, original_price, discount_price, 
              star_rating, review_count, data_info, link]
    
    print(result)
    driver.close()
    return result

For testing 

In [177]:
link = "https://cellphones.com.vn/samsung-galaxy-a54-8gb-256gb.html"
link1 = "https://cellphones.com.vn/iphone-15-plus.html"
link2 = "https://cellphones.com.vn/vivo-v25-pro.html"

driver = init_driver(link2)
cuu = save_link_to_product(driver, link2, "Oppo Reno 11F")

['Oppo Reno 11F', 'Oppo', '7690000', '13990000', '5.0/5', '4', {'Kích thước màn hình': '6.56 inches', 'Công nghệ màn hình': 'AMOLED', 'Camera sau': 'Camera chính 64MP OIS + Góc Rộng 8MP + Siêu Cận 2MP', 'Camera trước': '32MP AF, f/2.45', 'Chipset': 'MediaTek Dimensity 1300', 'Dung lượng RAM': '8GB + Mở rộng 8GB', 'Bộ nhớ trong': '128 GB', 'Pin': '4830 mAh', 'Thẻ SIM': '2 SIM (Nano-SIM)', 'Hệ điều hành': 'Android 12', 'Độ phân giải màn hình': '1080 x 2376 pixels (FullHD+)', 'Tính năng màn hình': 'Tần số quét 120 Hz, Độ sáng tối đa: 1300 nits, Kính cường lực Schott Xensation UP'}, 'https://cellphones.com.vn/vivo-v25-pro.html']


In [166]:
# links = df["Link"], name = df["Name"]
data = []

for index, row in df.iterrows():
    link, name = row["Link"], row["Title"]

    driver = init_driver(link)
    new_product = save_link_to_product(driver, link, name)

    data.append(new_product)

['iPhone 15 Pro Max 256GB | Chính hãng VN/A', 'iPhone', '30790000', '34990000', '5.0/5', '13', {'Kích thước màn hình': '6.7 inches', 'Công nghệ màn hình': 'Super Retina XDR OLED', 'Camera sau': 'Camera chính: 48MP, 24 mm, ƒ/1.78Camera góc siêu rộng: 12 MP, 13 mm, ƒ/2.2Camera Tele: 12 MP', 'Camera trước': '12MP, ƒ/1.9', 'Chipset': 'Apple A17 Pro 6 nhân', 'Dung lượng RAM': '8 GB', 'Bộ nhớ trong': '256 GB', 'Pin': '4422 mAh', 'Thẻ SIM': '2 SIM (nano‑SIM và eSIM)', 'Hệ điều hành': 'iOS 17', 'Độ phân giải màn hình': '2796 x 1290-pixel', 'Tính năng màn hình': 'Tốc độ làm mới 120Hz460 ppi2000 nitsHDRTrue ToneDải màu rộng (P3)Haptic TouchTỷ lệ tương phản 2.000.000:1'}, 'https://cellphones.com.vn/iphone-15-pro-max.html']
['iPhone 13 128GB | Chính hãng VN/A', 'iPhone', '14490000', '18990000', '5.0/5', '151', {'Kích thước màn hình': '6.1 inches', 'Công nghệ màn hình': 'Super Retina XDR OLED', 'Camera sau': 'Camera góc rộng: 12MP, f/1.6Camera góc siêu rộng: 12MP, ƒ/2.4', 'Camera trước': '12MP, f/2

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".block-same-product"}
  (Session info: chrome=122.0.6261.112)
Stacktrace:
	GetHandleVerifier [0x00007FF74B66AD22+56930]
	(No symbol) [0x00007FF74B5DF622]
	(No symbol) [0x00007FF74B4942E5]
	(No symbol) [0x00007FF74B4D98ED]
	(No symbol) [0x00007FF74B4D9A2C]
	(No symbol) [0x00007FF74B51A967]
	(No symbol) [0x00007FF74B4FBCDF]
	(No symbol) [0x00007FF74B5181E2]
	(No symbol) [0x00007FF74B4FBA43]
	(No symbol) [0x00007FF74B4CD438]
	(No symbol) [0x00007FF74B4CE4D1]
	GetHandleVerifier [0x00007FF74B9E6AAD+3709933]
	GetHandleVerifier [0x00007FF74BA3FFED+4075821]
	GetHandleVerifier [0x00007FF74BA3817F+4043455]
	GetHandleVerifier [0x00007FF74B709756+706710]
	(No symbol) [0x00007FF74B5EB8FF]
	(No symbol) [0x00007FF74B5E6AE4]
	(No symbol) [0x00007FF74B5E6C3C]
	(No symbol) [0x00007FF74B5D68F4]
	BaseThreadInitThunk [0x00007FFD7377257D+29]
	RtlUserThreadStart [0x00007FFD741EAA58+40]


In [ ]:
product_info_header = \
    ["product_name", "brand_name", "discount_price", "original_price",
    "rating", "review_count", "specs_info", "link"]

df = pd.DataFrame(data, columns=product_info_header)
df.to_csv("data/product_info.csv", header=True, index=False)